# Using trained tile detectors to extract feature maps

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import tensorflow as tf
import functools
import sys
home = os.path.expanduser("~")
user = home.split('/')[-1]
sys.path.append(home + '/alaska_github/src/')
from tools.tf2onnx_utils import *
from tools.models import *
from tools.train_estimator import *
from tools.jpeg_utils import *
from tools.tf_utils import *
import pickle
from tqdm import tqdm
from scipy.special import softmax

In [4]:
QF = 95
checkpoint_prefix = home + '/alaska_github/models/feature_extractors/QF'+str(QF)+'/tensorflow/'
checkpoint_path = checkpoint_prefix + 'model.cpkt-color_separated'
data_prefix = home + '/alaska_github/data/QF'+str(QF)+'/arbitrary_size/'
data_dict = {'COVER':data_prefix+'JPEG/COVER/',
            'JUNI':data_prefix+'JPEG/JUNI/',
            'UED':data_prefix+'JPEG/UED/',
            'NSF5':data_prefix+'JPEG/NSF5/',
            'EBS':data_prefix+'JPEG/EBS/'}
output_dict = {'COVER':data_prefix+'FEATURE_MAPS/COVER/',
            'JUNI':data_prefix+'FEATURE_MAPS/JUNI/',
            'UED':data_prefix+'FEATURE_MAPS/UED/',
            'NSF5':data_prefix+'FEATURE_MAPS/NSF5/',
            'EBS':data_prefix+'FEATURE_MAPS/EBS/'}

In [7]:
model_class = SR_net_feature_extractor_beast
tf.reset_default_graph()
input_image = tf.placeholder(tf.float32, shape=[1, None, None, 3], name='input')
global_step = tf.get_variable('global_step', dtype=tf.int32, shape=[], initializer=tf.constant_initializer(0), trainable=False)
featuremaps = model_class(input_image,  tf.estimator.ModeKeys.PREDICT)
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
saver_all = tf.train.Saver(max_to_keep=10000)
with tf.Session() as sess:
    sess.run(init_op)
    saver_all.restore(sess, checkpoint_path)
    for key in data_dict.keys():
        im_names = os.listdir(data_dict[key])
        if not os.path.exists(output_dict[key]):
            os.makedirs(output_dict[key], 0o755)
        for im_name in tqdm(im_names):
            tmp = jpeglib.jpeg(data_dict[key]+im_name, verbosity=0)
            image = decompress(tmp)
            f = sess.run(featuremaps, feed_dict = {input_image: image})
            f_dict = dict()
            for i,branch in enumerate(['YCrCb', 'CrCb', 'Y', 'Cr', 'Cb']):
                f_dict[branch] = f[i,:,:]
            with open(output_dict[key]+im_name.split('.jpg')[0]+'.p', 'wb') as handle:
                pickle.dump(f_dict, handle)



Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/yyousfi1/alaska_github/models/feature_extractors/QF95/tensorflow/model.cpkt-color_separated


100%|██████████| 1/1 [00:00<00:00,  4.29it/s]


These feature maps can now be used to train a classifier. In [1],the authors chose a MLP with 2 Hidden  Layers.

We encourage trying different classifiers (e.g. from scikit-learn libreary)

# Using frozen MLP to steganalyze

The frozen MLP provided in this repository corresponds to row 10 in Table 3 in [1].

In [80]:
graph = load_graph(home + '/alaska_github/models/detectors/QF95/tensorflow/MLP_frozen.pb')
# Checking the operations names
for op in graph.get_operations():
    print(op.name)

prefix/Placeholder
prefix/Flatten/flatten/Reshape/shape
prefix/Flatten/flatten/Reshape
prefix/layer_1/weights
prefix/layer_1/weights/read
prefix/layer_1/biases
prefix/layer_1/biases/read
prefix/layer_1/MatMul
prefix/layer_1/BiasAdd
prefix/layer_1/Relu
prefix/layer_2/weights
prefix/layer_2/weights/read
prefix/layer_2/biases
prefix/layer_2/biases/read
prefix/layer_2/MatMul
prefix/layer_2/BiasAdd
prefix/layer_2/Relu
prefix/layer_3/weights
prefix/layer_3/weights/read
prefix/layer_3/biases
prefix/layer_3/biases/read
prefix/layer_3/MatMul
prefix/layer_3/BiasAdd
prefix/output


In [81]:
x = graph.get_tensor_by_name('prefix/Placeholder:0')
y = graph.get_tensor_by_name('prefix/output:0')

In [90]:
test_file = output_dict['JUNI']+'iPAD-pro-7.1-13inch_2065.jpg'.split('.jpg')[0]+'.p'
with open(test_file, 'rb') as handle:
    test_feature_map = pickle.load(handle)
branches = ['Y' ,'YCrCb', 'CrCb']   
stego_schemes = ['EBS', 'JUNI', 'NSF5', 'UED']
test_feature_map = np.expand_dims(np.concatenate([test_feature_map[branch].reshape(-1) for branch in branches]),0)
with tf.Session(graph=graph) as sess:
    logits = sess.run(y, feed_dict={x: test_feature_map})
    
print('predicted as:', stego_schemes[np.argmax(softmax(logits))])

predicted as: JUNI


# References
[1] Yousfi, Yassine, et al. "Breaking ALASKA: Color separation for steganalysis in JPEG domain." Proceedings of the ACM Workshop on Information Hiding and Multimedia Security. ACM, 2019.